In [1]:
from tqdm import tqdm
import pandas as pd
from openpyxl import load_workbook
from io import BytesIO
import urllib
import warnings
import json
import re
from google.colab import drive
warnings.filterwarnings("ignore")
#from ipynb.fs.full.LIHTC_Page1_extraction import folder_list, construct_drive_url, load_workbook_from_url, get_year_by_name

In [2]:
#!pip install google-auth
from google.colab import auth
auth.authenticate_user()  # must authenticate

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# @title ### Function definitions
def construct_drive_url (file_id):
  return f"https://drive.google.com/uc?export=download&id={file_id}"

def load_workbook_from_url(url):
  """ https://stackoverflow.com/a/64725882 """
  file = urllib.request.urlopen(url).read()
  return load_workbook(filename = BytesIO(file), data_only=True)

def get_sheet_name_funds(year):
  if int(year) > 2020:
    return "Part II-Sources of Funds"
  else:
    return "Part III-Sources of Funds"

#gets the year given the folder name, eg:
def get_year_by_name(folder_name):
  return re.findall(r'\d{4}$', folder_name)[0]

#list all [ids, name] of files directly under folder folder_id
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [[f['id'], f['name']] for f in res['files']]

def get_user_selection(section_name):
  if section_name == "CONSTRUCTION FINANCING":
    return {"Mortgage A", "Mortgage B", "Mortgage C",
            "Federal Grant", "State, Local, or Private Grant", "Deferred Developer Fees", "Federal Housing Credit Equity",
            "State Housing Credit Equity", "Other Type (specify)", "Total Construction Period Costs from Development Budget:"}
  if section_name == "PERMANENT FINANCING":
    return {"Mortgage A (Lien Position 1)", "Mortgage B (Lien Position 2)", "Mortgage C (Lien Position 3)",
            "Other:", "Foundation, charity or other govt*", "Deferred Devlpr Fee", "Federal Grant",
            "State, Local, or Private Grant", "Federal Housing Credit Equity", "State Housing Credit Equity",
            "Historic Credit Equity", "Invstmt Earnings: T-E Bonds", "Invstmt Earnings: Taxable Bonds",
            "Income from Operations", "Total Permanent Financing:"}
  return {}

def get_boundaries(f_id, section_name):
  sections_file = '/content/drive/MyDrive/lihtc/sources_of_funds_sections.json'
  with open(sections_file, 'r') as file:
    sections = json.load(file)
  return sections[f_id][section_name]


#get all the entitites within a row
# iterate through all the cells in the row and get all the colored ones after the value
def get_entities_for_selections(ws,row_num, row, user_selections):
  data = []
  selection_row = {}
  key = None
  for cell in ws[row_num]:
    if cell.value in user_selections:
      key = cell.value
      user_selections.remove(key)
    if cell.fill.start_color.index == 41 and key:
      data.append(cell.value)
    if cell.fill.start_color.index == 'FFCCFFFF' and key:
      data.append(row[cell.column - 1])
    if cell.fill.start_color.index == 'FFCCFFCC' and key:
      data.append(row[cell.column - 1])
    if cell.fill.start_color.index == 42 and key:
      data.append(row[cell.column - 1])
    if cell.font.bold and key and key == 'Total Permanent Financing:':
      data.append(row[cell.column - 1])
  selection_row[key] = data
  return selection_row

def get_data_for_sheet (wb, sheet_name, user_selections,
                        section_start_row, section_end_row):
  ws = wb[sheet_name]
  data = {}
  for row_num, row in enumerate(ws.iter_rows(min_row=section_start_row, \
                                             max_row=section_end_row, \
                                             values_only=True),
                                start=section_start_row):
    if user_selections.intersection(row):
      data.update(get_entities_for_selections(ws, row_num, row, user_selections))
  return data


def get_data_by_drive_id(drive_id, sheet_name, year,
                         boundaries, section_name):
  wb = load_workbook_from_url (construct_drive_url (drive_id))
  user_selections = get_user_selection(section_name)
  data = get_data_for_sheet (wb, sheet_name, user_selections, boundaries[0],
                             boundaries[1])
  if section_name == "CONSTRUCTION FINANCING":
    standardize_dict_comstruction_funds(data)
  if section_name == "PERMANENT FINANCING":
    standardize_dict_permanent_funds(data)
  return data

In [5]:
# @title ### Standardization functions
def standardize_dict_comstruction_funds(data):
  for key, val in data.items():
    if len(val) == 4:
      dict_vals = {}
      dict_vals["Name of Financing Entity"] = val[0]
      dict_vals["Amount"] = val[1]
      dict_vals["Effective Interest Rate"] = val[2]
      dict_vals["Term (In Months)"] = val[3]
      data[key] = dict_vals
    if len(val) == 2:
      dict_vals = {}
      dict_vals["Name of Financing Entity"] = val[0]
      dict_vals["Amount"] = val[1]
      data[key] = dict_vals
    if len(val) == 3:
      dict_vals = {}
      dict_vals["Type"] = val[0]
      dict_vals["Name of Financing Entity"] = val[1]
      dict_vals["Amount"] = val[2]
      data[key] = dict_vals
    if len(val) == 1:
      data[key] = val[0]

def standardize_dict_permanent_funds(data):
  for key, val in data.items():
    if len(val) == 7:
      dict_vals = {}
      dict_vals["Name of Financing Entity"] = val[0]
      dict_vals["Amount"] = val[1]
      dict_vals["Effective Interest Rate"] = val[2]
      dict_vals["Term (Years)"] = val[3]
      dict_vals["Amort. (Years)"] = val[4]
      dict_vals["Loan Type"] = val[5]
      dict_vals["Annual Debt Service in Year One"] = val[6]
      data[key] = dict_vals
    if len(val) == 8:
      dict_vals = {}
      dict_vals["Type"] = val[0]
      dict_vals["Name of Financing Entity"] = val[1]
      dict_vals["Amount"] = val[2]
      dict_vals["Effective Interest Rate"] = val[3]
      dict_vals["Term (Years)"] = val[4]
      dict_vals["Amort. (Years)"] = val[5]
      dict_vals["Loan Type"] = val[6]
      dict_vals["Annual Debt Service in Year One"] = val[7]
      data[key] = dict_vals
    if len(val) == 2:
      dict_vals = {}
      dict_vals["Name of Financing Entity"] = val[0]
      dict_vals["Amount"] = val[1]
      data[key] = dict_vals
    if len(val) == 3:
      dict_vals = {}
      dict_vals["Type"] = val[0]
      dict_vals["Name of Financing Entity"] = val[1]
      dict_vals["Amount"] = val[2]
      data[key] = dict_vals
    if len(val) == 1:
      data[key] = val[0]

In [ ]:
# @title ### Read all the entities given the selection from a given file { run: "auto" }
# @markdown ---
drive_id = "11CbFZbm4qPtPcVwXsbLIdHpyyd2YrKH_" # @param {type:"string"}
sheet_name = "Part III-Sources of Funds" # @param {type:"string"}
section_name = "PERMANENT FINANCING" # @param ["CONSTRUCTION FINANCING", "PERMANENT FINANCING"]
year = "2019" # @param {type:"string"}
# @markdown ---
section_boundaries = get_boundaries(drive_id, section_name)
print(get_data_by_drive_id(drive_id, sheet_name, year,
                           section_boundaries, section_name))

{'Mortgage A (Lien Position 1)': {'Name of Financing Entity': 'HUD 221(d)(4) First Mortgage', 'Amount': 20100000, 'Effective Interest Rate': 0.0335, 'Term (Years)': 35, 'Amort. (Years)': 35, 'Loan Type': 'Amortizing', 'Annual Debt Service in Year One': 976008.3961278418}, 'Mortgage B (Lien Position 2)': {'Name of Financing Entity': None, 'Amount': None, 'Effective Interest Rate': None, 'Term (Years)': None, 'Amort. (Years)': None, 'Loan Type': None, 'Annual Debt Service in Year One': ''}, 'Mortgage C (Lien Position 3)': {'Name of Financing Entity': None, 'Amount': None, 'Effective Interest Rate': None, 'Term (Years)': None, 'Amort. (Years)': None, 'Loan Type': None, 'Annual Debt Service in Year One': ''}, 'Other:': {'Type': None, 'Name of Financing Entity': None, 'Amount': None, 'Effective Interest Rate': None, 'Term (Years)': None, 'Amort. (Years)': None, 'Loan Type': None, 'Annual Debt Service in Year One': ''}, 'Deferred Devlpr Fee': {'Name of Financing Entity': 'Gainesville Leased 

In [ ]:
# @title ### Get all the entities for all the files for a given year
drive_dir_id = "1_4mt1P6bumhc6ER3Y6q1DK6KelZtkx7D" # @param {type:"string"}
sheet_name = "Part III-Sources of Funds" # @param {type:"string"}
section_name = "CONSTRUCTION FINANCING" # @param ["CONSTRUCTION FINANCING", "PERMANENT FINANCING"]
year = "2015" # @param {type:"string"}

#get data for the year for all the fields in the section
def get_data_for_year(drive_dir_id, sheet_name,
                      year, section_name):
  data = []
  for file_info in tqdm (folder_list (drive_dir_id)):
    f_id = file_info[0]
    f_name = file_info[1]
    print("f name is: " + f_name)
    boundaries = get_boundaries(f_id, section_name)
    data_file = get_data_by_drive_id(file_info[0], sheet_name,
                                     year, boundaries, section_name)
    data_file["gdrive_id"] = f_id
    data_file["file_id"] = f_name
    data.append(data_file)
  return data
get_data_for_year(drive_dir_id, sheet_name,year, section_name)

In [ ]:
drive_dir_id = "1uN65Fi2GzhTb42AJhTaraDXHUhNag7Xa" # @param {type:"string"}
section_name = "PERMANENT FINANCING" # @param ["CONSTRUCTION FINANCING", "PERMANENT FINANCING"]
json_file_path = '/content/drive/MyDrive/lihtc/permanent_financing.json' # @param {type:"string"}

# read all files for all years
def write_all_files_to_json(drive_dir_id, section_name):
  data = []
  for file_info in tqdm (folder_list (drive_dir_id)):
    year_str = get_year_by_name(file_info[1])
    print(year_str)
    yearly_data = get_data_for_year(file_info[0], get_sheet_name_funds(year_str),
                                    year_str, section_name)
    data = data + yearly_data
  with open(json_file_path, 'w') as json_file:
   json.dump(data, json_file)
write_all_files_to_json(drive_dir_id, section_name)

  0%|          | 0/6 [00:00<?, ?it/s]

2015



  0%|          | 0/7 [00:00<?, ?it/s]

f name is: 2015-509wessblckgoracore.xlsx



 14%|█▍        | 1/7 [00:08<00:49,  8.23s/it]

...


 98%|█████████▊| 78/80 [18:18<00:28, 14.17s/it]

f name is: 2020-502lkwdcore.xlsx



 99%|█████████▉| 79/80 [18:32<00:14, 14.03s/it]

f name is: 2020-501pksdawcore.xlsx



100%|██████████| 6/6 [55:26<00:00, 554.48s/it]


In [ ]:
# @title ### This whole block is to write sections per sheet

sections_file = '/content/drive/MyDrive/lihtc/sources_of_funds_sections.json'
drive_dir_id = "1uN65Fi2GzhTb42AJhTaraDXHUhNag7Xa"

#writes sections for each file into a json file for all the files
def write_sections_for_all_years(drive_dir_id):
  sections = {}
  for file_info in tqdm (folder_list (drive_dir_id)):
    year = int(get_year_by_name(file_info[1]))
    years_section = get_section_for_year(file_info[0], get_sheet_name_funds(year))
    sections.update(years_section)
  with open(sections_file, 'w') as file:
    json.dump(sections, file)

#given a drive id and sheet name, we return the sections for the year
def get_section_for_year(drive_dir_id, sheet_name):
  data = {}
  for file_info in tqdm (folder_list (drive_dir_id)):
    wb = load_workbook_from_url (construct_drive_url (file_info[0]))
    ws = wb[sheet_name]
    data[file_info[0]] = get_sections_for_sheet(ws)
  return data

#given a sheet, we return the section bounaries for the sources of funds page
def get_sections_for_sheet(ws):
  sections = {}
  is_search_section = False

  for row_num, row in enumerate(ws.iter_rows(values_only=True),
                                start=1):
    if is_start_of_construction_finance_section(row):
      sections["CONSTRUCTION FINANCING"] = [row_num]
    if is_end_of_construction_finance_section(row):
      sections["CONSTRUCTION FINANCING"].append(row_num)
    if is_start_of_permanent_finance_section(row):
      sections["PERMANENT FINANCING"] = [row_num]
    if is_end_of_permanent_finance_section(row):
      sections["PERMANENT FINANCING"].append(row_num)

  return sections

def is_start_of_construction_finance_section(row):
  return "CONSTRUCTION FINANCING" in row

def is_end_of_construction_finance_section(row):
  return "PERMANENT FINANCING" in row

def is_start_of_permanent_finance_section(row):
  return "PERMANENT FINANCING" in row

def is_end_of_permanent_finance_section(row):
  return ("APPLICANT COMMENTS AND CLARIFICATIONS" in row)

In [ ]:
write_sections_for_all_years(drive_dir_id)

In [ ]:
import random

def pick_random_entities(lst, num_entities):
    if num_entities > len(lst):
        raise ValueError("Number of entities to pick exceeds the length of the list.")
    return random.sample(lst, num_entities)

# Example usage:
with open(json_file_path, 'r') as f:
      data = json.load(f)
my_list = data
random_entities = pick_random_entities(my_list, 11)
for rando in random_entities:
  print(rando['gdrive_id'])
  print(rando['file_id'])
  print(rando)

NameError: name 'json_file_path' is not defined